# Working with ISIC Images
Images are at the heart of the ISIC Archive, and this notebook
will give you a sense of what the ```isicarchive``` package
can do in terms of retrieving images from the archive, and the
kinds of methods you can apply to them.
## Preparation
We'll start again by creating an ```IsicApi``` object, which
allows us to access the various pieces of information and images
stored in the archive. The call to ```cache_images()``` will
then download information (and meta information) about all
images that are available--if you are logged in as a regular
user, you may have access to additional images! **If you're
using the ```cache_images()``` method, please use a different
cache folder if you want to work with both publicly available
and private images in separate sessions.**

Next, we'll select all images from the ```IsicApi``` object that
have a specific diagnosis, and then display those images.

In [ ]:
# Settings (please adapt to your needs!)
username = 'weberj3@mskcc.org'

# DO NOT PUT A PASSWORD into the document, it will be requested,
# or you can store it in a privately readable ~/.netrc file!

cache_folder = 'C:\\Users\\weberj3\\Documents\\ISIC'
# cache_folder = '/Data/MSKCC/isicarchive/cache'

# setting debug flag, this will print out every web-API operation
debug = False

# Preparation
from isicarchive.api import IsicApi
from isicarchive import func, imfunc
api = IsicApi(username, cache_folder=cache_folder, debug=debug)

In [ ]:
# caching images and segmentations; this can take a long time!!
# so you may want to skip this cell for now...
api.cache_images()
api.cache_segmentations()

In [ ]:
# list available studies (depends on user authentication!)
api.studies

In [ ]:
# load a study and cache the images associated with it
study = api.study('ISIC Annotation Study - All Features')

In [ ]:
# select images from this study
selection = api.select_images(['meta.clinical.melanocytic', '==', True], study.images)

In [ ]:
# load image and display image data
image = api.image(study.images[0])
image.load_image_data()
image.show_in_notebook()

In [ ]:
# load image's superpixel information and map array
image.load_superpixels()
image.map_superpixels()

In [ ]:
# create superpixel outlines as SVG paths
superpixel_colors = func.superpixel_colors()
outline_svg = imfunc.superpixel_outlines(
    image.superpixels['map'],
    image.superpixels['idx'].shape,
    out_format='osvg',
    path_attribs=superpixel_colors)
outlines = imfunc.superpixel_outlines(
    image.superpixels['map'],
    image.superpixels['idx'].shape,
    out_format='osvgp',
    path_attribs=superpixel_colors)

# display one path
outlines[0]

In [ ]:
# create heatmap with default settings
(heatmap, stats) = study.image_heatmap(study.images[0],mix_colors=False,underlay_gray=0.5)
api.display_image(heatmap)

In [ ]:
# display stats on the annotation features
stats['featnum']

In [ ]:
# retrieve segmentation (supports image name, image ID (via additional request), and segmentation ID)
segmentation = api.segmentation('ISIC_0000000')

In [ ]:
# get superpixels in image mask
sp_weights = segmentation.superpixels_in_mask()
sp_half_in = segmentation.superpixels_in_mask(0.5)
sp_full_in = segmentation.superpixels_in_mask(1.0)

# print sum
print('Number of superpixels: ' + str(len(sp_weights)))
print('Average coverage: ' + str(sum(sp_weights) / len(sp_weights)))
print('Superpixels at least half in mask: ' + str(len(sp_half_in)))
print('Superpixels fully in mask: ' + str(len(sp_full_in)))